# Compile and Deploy the pretrained PyTorch model from model zoo with SageMaker Neo

---

***[주의] 본 모듈은 PyTorch EIA 1.3.1 버전이나 1.5.1 버전에서 훈련을 수행한 모델만 배포가 가능합니다. 코드가 정상적으로 수행되지 않는다면, 프레임워크 버전을 동일 버전으로 맞춰 주시기 바랍니다.***

본 모듈에서는 Elastic Inference Accelerator(EIA)를 사용하여 모델을 배포해 보겠습니다.

## Elastic Inference Accelerator
훈련 인스턴스와 달리 실시간 추론 인스턴스는 계속 상시로 띄우는 경우가 많기에, 딥러닝 어플리케이션에서 low latency를 위해 GPU 인스턴스를 사용하면 많은 비용이 발생합니다.

Amazon Elastic Inference는 저렴하고 메모리가 작은 GPU 기반 가속기를 Amazon EC2, Amazon ECS, Amazon SageMaker에 연결할 수 있는 서비스로, Accelerator가 CPU 인스턴스에 프로비저닝되고 연결됩니다. EIA를 사용하면 GPU 인스턴스에 근접한 퍼포먼스를 보이면서 인스턴스 실행 비용을 최대 75%까지 절감할 수 있습니다. 

모든 Amazon SageMaker 인스턴스 유형, EC2 인스턴스 유형 또는 Amazon ECS 작업을 지원하며, 대부분의 딥러닝 프레임워크를 지원하고 있습니다. 지원되는 프레임워크 버전은 AWS CLI로 확인할 수 있습니다.

```bash
aws ecr list-images --repository-name tensorflow-inference-eia --registry-id 763104351884
aws ecr list-images --repository-name pytorch-inference-eia --registry-id 763104351884
aws ecr list-images --repository-name mxnet-inference-eia --registry-id 763104351884
```

- 참조: https://aws.amazon.com/ko/blogs/korea/amazon-elastic-inference-gpu-powered-deep-learning-inference-acceleration/

## TorchScript Compile (Tracing)

PyTorch 프레임워크에서 EI를 사용하기 위해서는 [TorchScript](https://pytorch.org/docs/1.3.1/jit.html)로 모델을 컴파일해야 하며, 2022년 2월 시점에서는 PyTorch 1.3.1와 PyTorch 1.5.1을 지원하고 있습니다. TorchScript는 PyTorch 코드에서 직렬화 및 최적화 가능한 모델로 컴파일하며 Python 인터프리터의 글로벌 인터프리터 잠금 (GIL)과 무관하기 때문에 Python 외의 언어에서 로드 가능하고  최적화가 용이합니다.

TorchScript로 변환하는 방법은 **tracing** 방식과 **scripting** 방식이 있으며, 본 핸즈온에서는 tracing 방식을 사용하겠습니다.
참고로 tracing 방식은 샘플 입력 데이터를 모델에 입력 후 그 입력의 흐름(feedforward)을 기록하여 포착하는 메커니즘이며, scripting 방식은 모델 코드를 직접 분석해서 컴파일하는 방식입니다.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, sagemaker, random

import torch
print(sagemaker.__version__)
print(torch.__version__)

<br>

## 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트를 저장합니다. 

In [ ]:
%%writefile src/infer_pytorch_eia.py

import io
import json
import logging
import os
import pickle

import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image  # Training container doesn't have this package

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
    
# To use new EIA inference API, customer should use attach_eia(model, eia_ordinal_number)
VERSIONS_USE_NEW_API = ["1.5.1"]

def model_fn(model_dir):
    try:
        loaded_model = torch.jit.load("model.pth", map_location=torch.device("cpu"))
        if torch.__version__ in VERSIONS_USE_NEW_API:
            import torcheia

            loaded_model = loaded_model.eval()
            loaded_model = torcheia.jit.attach_eia(loaded_model, 0)
        return loaded_model
    except Exception as e:
        logger.exception(f"Exception in model fn {e}")
        return None

def transform_fn(model, payload, request_content_type='application/octet-stream', 
                 response_content_type='application/json'):

    logger.info('Invoking user-defined transform function')

    if request_content_type != 'application/octet-stream':
        raise RuntimeError(
            'Content type must be application/octet-stream. Provided: {0}'.format(request_content_type))

    # preprocess
    decoded = Image.open(io.BytesIO(payload))
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[
                0.485, 0.456, 0.406], std=[
                0.229, 0.224, 0.225]),
    ])
    normalized = preprocess(decoded)
    batchified = normalized.unsqueeze(0)

    # predict
    # With EI, client instance should be CPU for cost-efficiency. Subgraphs with unsupported arguments run locally. Server runs with CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batchified = batchified.to(device)
    
    # Please make sure model is loaded to cpu and has been eval(), in this example, we have done this step in model_fn()
    with torch.no_grad():
        if torch.__version__ in VERSIONS_USE_NEW_API:
            # Please make sure torcheia has been imported
            import torcheia

            # We need to set the profiling executor for EIA
            torch._C._jit_set_profiling_executor(False)
            with torch.jit.optimized_execution(True):
                result =  model.forward(batchified)
        # Set the target device to the accelerator ordinal
        else:
            with torch.jit.optimized_execution(True, {"target_device": "eia:0"}):
                result = model(batchified)

    # Softmax (assumes batch size 1)
    result = np.squeeze(result.detach().cpu().numpy())
    result_exp = np.exp(result - np.max(result))
    result = result_exp / np.sum(result_exp)

    response_body = json.dumps(result.tolist())

    return response_body, response_content_type

<br>

## 2. Import pre-trained model from TorchVision
---
본 예제는 TorchVision의 pre-trained 모델 중 MnasNet을 사용합니다.
MnasNet은 정확도(accuracy)와 모바일 디바이스의 latency를 모두 고려한 강화학습 기반 NAS(neural architecture search)이며, TorchVision은 image classification에 최적화된 MNasNet-B1을 내장하고 있습니다. 
(참조 논문: https://arxiv.org/pdf/1807.11626.pdf)

In [ ]:
import torch
import torchvision.models as models
import tarfile

model = models.mnasnet1_0(pretrained=True)

input_shape = [1,3,224,224]
traced_model = torch.jit.trace(model.float().eval(), torch.zeros(input_shape).float())
torch.jit.save(traced_model, 'model.pth')

### Local Inference without Endpoint

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있기 때문에, 로컬 환경 상에서 추론을 수행하면서 디버깅하는 것을 권장합니다. 아래 코드 셀의 코드를 예시로 참조해 주세요.

In [ ]:
def get_inference(img_path, predictor, show_img=True):
    with open(img_path, mode='rb') as file:
        payload = bytearray(file.read())

    response = predictor.predict(payload)
    result = json.loads(response.decode())
    pred_cls_idx, pred_cls_str, prob = parse_result(result, show_img)
    
    return pred_cls_idx, pred_cls_str, prob 


def parse_result(result, img_path, show_img=True):
    pred_cls_idx = np.argmax(result)
    pred_cls_str = label_map[str(pred_cls_idx)]
    prob = np.amax(result)*100
    
    if show_img:
        import matplotlib.pyplot as plt
        img = Image.open(img_path)
        plt.figure()
        fig, ax = plt.subplots(1, figsize=(10,10))
        ax.imshow(img)
        overlay_text = f'{pred_cls_str} {prob:.2f}%'
        ax.text(20, 40, overlay_text, style='italic',
                bbox={'facecolor': 'yellow', 'alpha': 0.5, 'pad': 10}, fontsize=20)

    return pred_cls_idx, pred_cls_str, prob

모델 배포가 완료되었으면, 저자가 직접 준비한 샘플 이미지들로 추론을 수행해 보겠습니다.

In [ ]:
import os
import json
import numpy as np
from io import BytesIO
from PIL import Image
from src.infer_pytorch_eia import transform_fn

path = "./samples"
img_list = os.listdir(path)
img_path_list = [os.path.join(path, img) for img in img_list]

#test_idx = random.randint(0, len(img_list)-1)
test_idx = 0
img_path = img_path_list[test_idx]

with open(img_path, mode='rb') as file:
    payload = bytearray(file.read())

클래스 인덱스에 대응하는 클래스명을 매핑하기 위한 딕셔너리를 생성합니다.

In [ ]:
from src.utils import get_label_map_imagenet
label_file = 'metadata/imagenet1000_clsidx_to_labels.txt'
label_map = get_label_map_imagenet(label_file)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = torch.jit.load('model.pth')
#model = model.to(device)

response_body, _ = transform_fn(traced_model, payload)
result = json.loads(response_body)
parse_result(result, img_path)

<br>

## 3. Compile the Model
---

### 모델 압축

In [ ]:
with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add('model.pth')

In [ ]:
import boto3
import sagemaker
import time
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
sm_sess = sagemaker.Session()
region = sm_sess.boto_region_name
bucket = sm_sess.default_bucket()

instance_type = "ml.m5.large"
accelerator_type = "ml.eia2.xlarge"

# TorchScript model
tar_filename = "model.tar.gz"

# You can also upload model artifacts to S3
# print('Upload tarball to S3')
# model_data = sagemaker_session.upload_data(path=tar_filename, bucket=bucket, key_prefix=prefix)
model_data = tar_filename

endpoint_name = (
    "mnist-ei-traced-{}-{}".format(instance_type, accelerator_type)
    .replace(".", "")
    .replace("_", "")
)

In [ ]:
pytorch_model = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point="infer_pytorch_eia.py",
    source_dir="src",
    framework_version="1.5.1",
    py_version="py3",
    sagemaker_session=sm_sess,
)

### Deployment

instance_type과 accelerator_type만 EIA에 적절하게 변경해 주시면 되며, 로컬 모드 배포도 가능합니다.

In [ ]:
# Attach EI remotely
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import IdentitySerializer

# Function will exit before endpoint is finished creating
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    accelerator_type=accelerator_type,
    endpoint_name=endpoint_name,
    serializer=IdentitySerializer(content_type='application/octet-stream'),
    deserializer=JSONDeserializer(),
    wait=False
)

# # Attach EI locally
# # Deploys the model to a local endpoint
# predictor = pytorch_model.deploy(
#     initial_instance_count=1,
#     instance_type='local',
#     accelerator_type='local_sagemaker_notebook',
#     serializer=IdentitySerializer(content_type='application/octet-stream'),
#     deserializer=JSONDeserializer()        
# )

### Wait for the endpoint jobs to complete

엔드포인트가 생성될 때까지 기다립니다. 약 5-10분의 시간이 소요됩니다.

In [ ]:
from IPython.core.display import display, HTML

def make_endpoint_link(region, endpoint_name, endpoint_task):
    
    endpoint_link = f'<b><a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">{endpoint_task} Review Endpoint</a></b>'   
    return endpoint_link 
        
endpoint_link = make_endpoint_link(region, predictor.endpoint_name, '[Deploy EIA model]')

display(HTML(endpoint_link))

In [ ]:
sm_sess.wait_for_endpoint(predictor.endpoint_name, poll=5)

<br>

## 4. Inference
---

모델 배포가 완료되었으면, 추론을 수행합니다.

In [ ]:
import os
import json
import numpy as np
from io import BytesIO
from PIL import Image

img_list = os.listdir(path)
img_path_list = [os.path.join(path, img) for img in img_list]
print(img_path_list)

In [ ]:
test_idx = random.randint(0, len(img_list)-1)
img_path = img_path_list[test_idx]

with open(img_path, mode='rb') as file:
    payload = bytearray(file.read())

response = predictor.predict(payload)
pred_cls_idx, pred_cls_str, prob = parse_result(response, img_path, show_img=True)

마지막으로 latency를 측정합니다.

In [ ]:
import time
start_time = time.time()
num_tests = 20
for _ in range(num_tests):
    response = predictor.predict(payload)
end_time = (time.time()-start_time)
print(f'EIA optimized inference time is {(end_time/num_tests)*1000:.4f} ms (avg)')

## Endpoint Clean-up

SageMaker Endpoint로 인한 과금을 막기 위해, 본 핸즈온이 끝나면 반드시 Endpoint를 삭제해 주시기 바랍니다.

In [ ]:
predictor.delete_endpoint()
pytorch_model.delete_model()